# This is the tutorials of using PyABSA for aspect term extraction
Drafted for v2.0 and higher versions. Note there are many breaking changes in v2.0, so you do not need to upgrade to v2.0 and higher versions if you are using code, API, checkpoints, datasets or anything from v1.0. Let's begin the introduction.

In [ ]:
!pip install pyabsa >= 2.0.0
from pyabsa import AspectTermExtraction as ATEPC

# ATEPCModelList
There are three types of APC models for aspect term extraction, which are based on the local context focus mechanism
Notice: when you select to use a model, please make sure to carefully manage the configurations, e.g., for glove-based models, you need to set hidden dim and embed_dim manually.
We already provide some pre-defined configurations. Refer to the source code if you have any question
e.g.,

In [ ]:
# config = ATEPC.ATEPCConfigManager.get_atepc_config_glove()  # get pre-defined configuration for GloVe model, the default embed_dim=300
config = ATEPC.ATEPCConfigManager.get_atepc_config_english()  # this config contains 'pretrained_bert', it is based on pretrained models


# ATEPCDatasetList
There are the [datasets](https://github.com/yangheng95/ABSADatasets) from publication or third-party contribution. There dataset can be downloaded and processed automatically.
In pyabsa, you can pass a set of datasets to train a model.
e.g., for using integrated datasets:


In [ ]:
from pyabsa import DatasetItem

dataset = ATEPC.ATEPCDatasetList.SemEval
# now the dataset is a DatasetItem object, which has a name and a list of subdatasets
# dataset contains Laptop14, Restaurant14, Restaurant16 datasets

You can use your own dataset provided that it is formatted according to [ABSADatasets](https://github.com/yangheng95/ABSADatasets#important-rename-your-dataset-filename-before-use-it-in-pyabsa)

In [ ]:
# Put your dataset into integrated_datasets folder, it this folder does not exist, you need to call:
# from pyabsa import download_all_available_datasets
# download_all_available_datasets()

to pass datasets to PyABSA trainers, you can

In [ ]:
my_dataset = DatasetItem('my_dataset', ['my_dataset1', 'my_dataset2'])
# my_dataset1 and my_dataset2 are the dataset folders. In there folders, the train dataset is necessary


# Training
Let's prepare to train

In [ ]:
from pyabsa import ModelSaveOption, DeviceTypeOption

trainer = ATEPC.ATEPCTrainer(
    config=config,
    dataset=dataset,
    from_checkpoint=None,
    # if you want to resume training from our pretrained checkpoints, you can pass the checkpoint name here
    auto_device=DeviceTypeOption.AUTO,
    path_to_save=None,  # set a path to save checkpoints, if it is None, save checkpoints at 'checkpoints' folder
    checkpoint_save_mode=ModelSaveOption.SAVE_MODEL_STATE_DICT,
    load_aug=False,
    # there are some augmentation dataset for integrated datasets, you use them by setting load_aug=True to improve performance
)


to load trained model for inference:

In [ ]:
sentiment_classifier = trainer.load_trained_model()
assert isinstance(sentiment_classifier, ATEPC.AspectExtractor)

# Inference

## Use our checkpoints to initialize a SentimentClassifier

In [ ]:
from pyabsa import available_checkpoints
ckpts = available_checkpoints()
# find a suitable checkpoint and use the name:
aspect_extractor = ATEPC.AspectExtractor(checkpoint='english')  # here I use the english checkpoint which is trained on all English datasets in PyABSA

## Simple Prediction

In [ ]:
atepc_examples = ['But the staff was so nice to us .',
                  'But the staff was so horrible to us .',
                  r'Not only was the food outstanding , but the little ` perks \' were great .',
                  'It took half an hour to get our check , which was perfect since we could sit , have drinks and talk !',
                  'It was pleasantly uncrowded , the service was delightful , the garden adorable , '
                  'the food -LRB- from appetizers to entrees -RRB- was delectable .',
                  'How pretentious and inappropriate for MJ Grill to claim that it provides power lunch and dinners !'
                  ]

for ex in atepc_examples:
    aspect_extractor.predict(
        text=ex,
        print_result=True,
        ignore_error=True,  # ignore an invalid example, if it is False, invalid examples will raise Exceptions
        eval_batch_size=32
    )


## Batch Inference

In [ ]:
sentiment_classifier.batch_predict(
    inference_source=None,
    print_result=True,
    save_result=False,
    ignore_error=True,
    eval_batch_size=32
)

# Annotate your own datasets via PyABSA
[Auto-Annotation](https://github.com/yangheng95/ABSADatasets#auto-annoate-your-datasets-via-pyabsa)  # available for v1.0 currently
[Manually-Annotation](https://github.com/yangheng95/ABSADatasets/tree/v1.2/DPT)

# Deploy a ATEPC demo
TBC ...
